In [1]:
from ase import Atoms
from ase.data import atomic_numbers, covalent_radii
from ase import neighborlist
from clus_utils import  addAtoms, fixOverlap, checkBonded, checkSimilar
from ase.calculators.emt import EMT
import numpy as np
from ase.optimize import BFGS
import random
from ase.io  import read, write, Trajectory
from symmetry_function import make_snn_params, wrap_symmetry_functions
import copy

In [2]:
def ase_to_list(clus):
        list_coord = []
        for i in range(len(clus)):
                elem, x, y, z = clus.get_chemical_symbols()[i],  clus.get_positions()[i][0], clus.get_positions()[i][1], clus.get_positions()[i][2]
                list_coord.append([elem, x, y , z])    
        clus = list_coord
        return clus

In [3]:
#

In [4]:
DIRECTION =[np.array([1,0,0]),
           np.array([-1,0,0]),
           np.array([0,1,0]),
           np.array([0,-1,0]),
           np.array([0,0,1]),
           np.array([0,0,-1]),
          ]

class ClusEnv():
    
    def __init__(self,
                 eleNames=None,
                 eleNums=None
                ):
        
        self.eleNames = eleNames
        self.eleNums= eleNums
        self.eleRadii = [covalent_radii[atomic_numbers[ele]] for ele in eleNames]
        self.seed = 100
    
    def gen_clus(self):
        ele_initial = [self.eleNames[0], self.eleNames[-1]]
        d = (self.eleRadii[0] + self.eleRadii[-1]) / 2
        clusm = Atoms(ele_initial, [(-d, 0.0, 0.0), (d, 0.0, 0.0)])
        clus = addAtoms(clusm, self.eleNames, self.eleNums, self.eleRadii, self.seed)
        clus = fixOverlap(clus)
        
        elements = np.array(clus.symbols)
        _, idx = np.unique(elements, return_index=True)
        elements = list(elements[np.sort(idx)])
        
        return clus, elements
    
    def get_initial_clus(self):
        clus, elements = self.gen_clus()

        
        Gs = {}
        Gs["G2_etas"] = np.logspace(np.log10(0.05), np.log10(5.0), num=4)
        Gs["G2_rs_s"] = [0] * 4
        Gs["G4_etas"] = [0.005]
        Gs["G4_zetas"] = [1.0]
        Gs["G4_gammas"] = [+1.0, -1]
        Gs["cutoff"] = 6.5

        G = copy.deepcopy(Gs)

            # order descriptors for simple_nn
        cutoff = G["cutoff"]
        G["G2_etas"] = [a / cutoff**2 for a in G["G2_etas"]]
        G["G4_etas"] = [a / cutoff**2 for a in G["G4_etas"]]
        descriptors = (
            G["G2_etas"],
            G["G2_rs_s"],
            G["G4_etas"],
            G["cutoff"],
            G["G4_zetas"],
            G["G4_gammas"],
        )
        snn_params = make_snn_params(elements, *descriptors)                
        return clus, snn_params
    
    
    def get_fingerprints(self,clus, snn_params):
    #get fingerprints from amptorch as better state space feature
        fps = wrap_symmetry_functions(clus, snn_params)
        fp_length = fps.shape[-1]
        return fps, fp_length

In [17]:
def test_fps():
    ele_Names = ['Ni']
    ele_Nums = [20]
    clus_example = ClusEnv(ele_Names, ele_Nums)
    clus_ex1,snn_params = clus_example.get_initial_clus()
    ene1 = clus_ex1.get_potential_energy()
    fps1, fp_len1 = clus_example.get_fingerprints(clus_ex1, snn_params) 
    #print(fps1, fp_len1)
    #print('\n')

    random.seed()
    i = random.randint(0,len(clus_ex1)-1)
    shift_move = random.choice(DIRECTION)
    #print(i,shift_move)

    clus_ex2 = clus_ex1.copy()
    clus_ex2[i].position =  clus_ex1.get_positions()[i] + shift_move * 2.5        
    clus_ex2.calc = EMT()
    dyn= BFGS(clus_ex2, logfile='test.log')
    dyn.run(fmax=0.02, steps=1000)
    ene2 = clus_ex2.get_potential_energy()

    fps2, fp_len2 = clus_example.get_fingerprints(clus_ex2, snn_params) 
    #print(fps2, fp_len2)
    #print('\n')
    relative_fps = fps2 - fps1
    max_relative_fps = np.max(relative_fps)
    #print(relative_fps)
    #print(np.max(relative_fps))

    diff_ene = ene2 - ene1
    diff_iner = clus_ex2.get_moments_of_inertia() - clus_ex1.get_moments_of_inertia()

    #print(ene1, ene2, diff_ene, diff_iner)

    print(diff_ene, diff_iner,checkSimilar(clus_ex1, clus_ex2), max_relative_fps, fp_len1, fp_len2)

In [18]:
for i in range(30):
    print('iteration', i)
    test_fps()
#rint(reward)

iteration 0
0.644848887705372 [ 563.95479801 -321.80124403  -47.87941859] False 9.68956634517056 6 6
iteration 1
-3.764700478114946e-05 [ 1.00846982 -1.57813607 -2.38829807] True 0.02954313844221268 6 6
iteration 2
-8.873110546758767e-05 [ 2.68003529 -2.62507166 -1.36729372] True 0.02685550775651535 6 6
iteration 3
-0.00013114751588361173 [ 2.6255446  -3.15800077 -2.93594791] True 0.02394232242682115 6 6
iteration 4
0.644848887705372 [ 563.95479801 -321.80124403  -47.87941859] False 9.68956634517056 6 6
iteration 5
-1.7571372417535258e-05 [ 0.89984141 -2.23304171 -2.37574782] True 0.02231188342319612 6 6
iteration 6
0.9616141335516453 [-135.99176402  525.94327769  990.43967235] False 0.9206960598434764 6 6
iteration 7
-0.00012252219131880793 [ 1.3532303  -1.79949048 -2.17186473] True 0.021993051192882973 6 6
iteration 8
1.8466680271698799 [ -90.09719587  823.4035673  1135.72974223] False 1.4620140401653767 6 6
iteration 9
2.620304771028259e-05 [ 2.78238859 -4.51683123 -2.83001925] True

In [14]:
x = -3.5
y = -4.5
print(-x, -y)

3.5 4.5
